In [1]:
# configuration for using Gemini API
import google.generativeai as genai
from google.ai.generativelanguage_v1beta.types import content
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
gemini_token = user_secrets.get_secret("GEMINI_API")
genai.configure(api_key=gemini_token)

In [2]:
# instruction and format configuration
sys_instruction = "Your task is to generate a list of sentences in both Arabic and English that could potentially cause confusion or misinterpretation for individuals unfamiliar with these languages. Focus on creating examples from the domains of medical and engineering. "

generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "response_schema": content.Schema(
    type = content.Type.OBJECT,
    required = ["Arabic", "English"],
    properties = {
      "Arabic": content.Schema(
        type = content.Type.STRING,
      ),
      "English": content.Schema(
        type = content.Type.STRING,
      ),
    },
  ),
  "response_mime_type": "application/json",
}

In [3]:
# initialize model
model = genai.GenerativeModel(
  model_name="gemini-2.0-flash-exp",
  generation_config=generation_config,
  system_instruction=sys_instruction,
)

response = model.generate_content("Generate")
print(response.text)

{
  "Arabic": "أجريت العملية على المريض وهو مستلق على بطنه",
  "English": "The operation was performed on the patient while he was lying on his stomach."
}
